# Гипотеза
#### Оценить количество роботов, которые заходили на наш сайт в период с 01.01.2016 по 01.07.2019 гг. 
#### Определить: 
* какие источники трафика содержат бОльшую долю роботов? 
* в какие часы суток чаще заходят роботы?

# Проверка гипотезы

In [1]:
import json # импортируем библиотеку JSON, т.к. в ответ на запрос от API данные придут в формате JSON
import requests #импортируем библиотеку requests, т.к. запрос мы будем делать через Python
from pprint import pprint # импортируем библиотеку pprint, чтобы в более аккуратном виде отображать ответ от API
token = '' #Вставьте здесь свой токен
headers = {'Authorization': 'OAuth ' + token}
params = {'metrics': 'ym:s:visits', 
          'dimensions': 'ym:s:hour,ym:s:isRobot,ym:s:lastsignTrafficSource',  
          'date1': '2016-01-01',  
          'date2': '2019-07-01',     
          'ids': 30177909,
          'accuracy': 'full',
          'limit': 100000}

response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers) #делаем запрос к API Яндекс.Метрики
print (response.status_code)
metrika_data = response.json()

200


In [2]:
import pandas as pd #импортируем библиотеку Pandas 
metrika_df = pd.DataFrame(metrika_data['data'])

In [3]:
# Используем функцию getMetrikaDataInListOfDicts(), чтобы избавиться от вложенных словарей
def getMetrikaDataInListOfDicts(metrika_data):  
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

metrika_df = pd.DataFrame(getMetrikaDataInListOfDicts(metrika_data))
display(metrika_df.head(10))

,ym:s:hour,ym:s:isRobot,ym:s:lastsignTrafficSource,ym:s:visits
0,15:00,People,Search engine traffic,3915.0
1,16:00,People,Search engine traffic,3862.0
2,14:00,People,Search engine traffic,3734.0
3,12:00,People,Search engine traffic,3728.0
4,11:00,People,Search engine traffic,3674.0
5,13:00,People,Search engine traffic,3598.0
6,17:00,People,Search engine traffic,3556.0
7,10:00,People,Search engine traffic,3204.0
8,18:00,People,Search engine traffic,3062.0
9,19:00,People,Search engine traffic,2651.0


In [4]:
metrika_df['ym:s:visits'] = metrika_df['ym:s:visits'].astype(int) #Из типа float64 преобразовываем данные в тип int

In [5]:
display(metrika_df[metrika_df['ym:s:lastsignTrafficSource'].isnull()]) #определим данные, для которых не указан источник трафика, делаем срез (slice) датафрейма

,ym:s:hour,ym:s:isRobot,ym:s:lastsignTrafficSource,ym:s:visits
154,19:00,People,None,9
155,22:00,People,None,9
157,09:00,People,None,8
158,17:00,People,None,8
166,10:00,People,None,5
171,22:00,Robots,None,5
179,13:00,Robots,None,4
183,19:00,Robots,None,4
184,20:00,People,None,4
187,01:00,Robots,None,3


In [6]:
metrika_df.columns = [col.replace('ym:s:','') for col in metrika_df.columns] #переименуем столбцы
print(metrika_df.columns) 

Index(['hour', 'isRobot', 'lastsignTrafficSource', 'visits'], dtype='object')


In [7]:
metrika_df['isRobot'].unique() #какие уникальные значения есть в столбце isRobot

array(['People', 'Robots'], dtype=object)

#### Наша гипотеза требует исследования из каких источников и в какое время суток чаще всего заходят роботы.
Построим сводную таблицу pivot_df с помощью функции pivot_table, в которой отобразим количество роботов, которые заходят на сайт:

In [8]:
pivot_df = metrika_df.pivot_table(columns='isRobot',values='visits',aggfunc='sum')
display(pivot_df)

isRobot,People,Robots
visits,66566,2798


In [9]:
print(metrika_df['hour'].min()) #найдем минимальное и максимальное время суток
print(metrika_df['hour'].max())

00:00
23:00


In [10]:
visits_by_robots = metrika_df[metrika_df['isRobot'] == 'Robots']  # Количество посещений роботами нашего сайта
display(visits_by_robots)

,hour,isRobot,lastsignTrafficSource,visits
64,15:00,Robots,Direct traffic,105
66,16:00,Robots,Direct traffic,103
68,11:00,Robots,Direct traffic,102
74,17:00,Robots,Direct traffic,88
75,02:00,Robots,Direct traffic,87
...,...,...,...,...
268,21:00,Robots,Social network traffic,1
270,22:00,Robots,Internal traffic,1
271,22:00,Robots,Link traffic,1
273,23:00,Robots,None,1


In [11]:
visits_by_robots_hour = visits_by_robots.groupby(['hour']).agg({'visits': 'sum'}) # Найдем долю посещений роботами по часам
visits_by_robots_hour.apply(lambda x:100 * x / float(x.sum())) 

,visits
hour,
00:00,3.466762
01:00,3.073624
02:00,3.752680
03:00,2.394568
04:00,3.073624
05:00,3.288063
06:00,2.609006
07:00,2.787706
08:00,3.431022


In [12]:
visits_by_robots_source = visits_by_robots.groupby(['lastsignTrafficSource']).agg({'visits': 'sum'}) # Найдем долю посещений роботами по источникам
visits_by_robots_source.apply(lambda x:100 * x / float(x.sum()))

,visits
lastsignTrafficSource,
Direct traffic,66.231411
Internal traffic,0.290170
Link traffic,1.124411
Search engine traffic,30.685528
Social network traffic,1.668480


# Выводы по нашей гипотезе:

 Итак, **количество роботов**, которые заходили на наш сайт в период с 01.01.2016 по 01.07.2019 гг. - **2777**.
 
 **Часы**, в которые наблюдается **наибольшая доля посещений сайта роботами** - **с 10 до 17 часов**.  
 
 **Источник**, содержащий бОльшую долю роботов - **Direct Traffic**.
 
 Наибольшее количество посещений сайта роботами из источника Direct Traffic наблюдалось в 2 часа утра и с 11:00 до 17:00.
 
 Гипотеза подтверждается в том, что основные посетители сайта - живые люди.